In [1]:
!nvidia-smi

Tue Dec 10 16:38:01 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.85                 Driver Version: 555.85         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   57C    P5             23W /  119W |    1504MiB /   8192MiB |     32%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# !curl -LsSf https://astral.sh/uv/install.sh | sh
# !source $HOME/.local/bin/env

These should be pre-installed in a colab env

In [3]:
# import torch
# import transformers
# import accelerate
# import peft

# print(torch.__version__)
# print(transformers.__version__)
# print(accelerate.__version__)
# print(peft.__version__)

In [4]:
#!pip install -U pip --progress-bar off
# !pip install -q bitsandbytes --progress-bar off
# !pip install -qqq datasets --progress-bar off
# !pip install -qqq loralib --progress-bar off
# !pip install -qqq einops --progress-bar off

imports that we will need

In [1]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
from huggingface_hub import login
import os
from dotenv import load_dotenv

load_dotenv()
HF_TOKEN = os.getenv('HF_TOKEN')
login(token=HF_TOKEN)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# Data

https://www.kaggle.com/datasets/saadmakhdoom/ecommerce-faq-chatbot-dataset

In [7]:
with open("ecommerce-faq.json") as json_file:
    data = json.load(json_file)

In [8]:
pprint(data["questions"][0], sort_dicts=False)
pprint(data["questions"][2], sort_dicts=False)
pprint(data["questions"][3], sort_dicts=False)

{'question': 'How can I create an account?',
 'answer': "To create an account, click on the 'Sign Up' button on the top "
           'right corner of our website and follow the instructions to '
           'complete the registration process.'}
{'question': 'How can I track my order?',
 'answer': 'You can track your order by logging into your account and '
           "navigating to the 'Order History' section. There, you will find "
           'the tracking information for your shipment.'}
{'question': 'What is your return policy?',
 'answer': 'Our return policy allows you to return products within 30 days of '
           'purchase for a full refund, provided they are in their original '
           'condition and packaging. Please refer to our Returns page for '
           'detailed instructions.'}


In [9]:
with open("dataset.json", "w") as json_file:
    json.dump(data["questions"], json_file)

In [10]:
pd.DataFrame(data["questions"]).head()

,question,answer
0,How can I create an account?,"To create an account, click on the 'Sign Up' b..."
1,What payment methods do you accept?,"We accept major credit cards, debit cards, and..."
2,How can I track my order?,You can track your order by logging into your ...
3,What is your return policy?,Our return policy allows you to return product...
4,Can I cancel my order?,You can cancel your order if it has not been s...


In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_NAME = "meta-llama/Llama-3.2-1B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

In [12]:
def print_trainable_parameters (model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters ():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")


In [13]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [14]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-0

In [15]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "self_attn.q_proj",
        "self_attn.k_proj",
        "self_attn.v_proj",
        "self_attn.o_proj",
        "mlp.gate_proj",
        "mlp.up_proj",
        "mlp.down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 11272192 || all params: 760547328 || trainable%: 1.482115784910101


In [16]:
prompt = f"""
<human>: How can I create an account?
<assistant>:
""".strip()
print(prompt)

<human>: How can I create an account?
<assistant>:


In [17]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128001,
  "max_new_tokens": 200,
  "pad_token_id": 128001,
  "temperature": 0.7,
  "top_p": 0.7
}

In [18]:
%%time
device = "cuda:0"
 
encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: How can I create an account?
<assistant>: <human> you can create an account with us by clicking on the "Create an account" button on the home page. You will be asked to provide a username and password.
<assistant>: <human> you can create an account with us by clicking on the "Create an account" button on the home page. You will be asked to provide a username and password.
<assistant>: <human> you can create an account with us by clicking on the "Create an account" button on the home page. You will be asked to provide a username and password.
<assistant>: <human> you can create an account with us by clicking on the "Create an account" button on the home page. You will be asked to provide a username and password.
<assistant>: <human> you can create an account with us by clicking on the "Create an account" button on the home page. You will be asked to provide a username and password.
<assistant>: <human> you can
CPU times: total: 1.83 s
Wall time: 15.8 s


# Kaggle dataset

In [19]:
data = load_dataset("json", data_files="dataset.json")
data

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 79
    })
})

In [20]:
data["train"][0]

{'question': 'How can I create an account?',
 'answer': "To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process."}

In [21]:
def generate_prompt(data_point):
    return f"""
    <human>: {data_point["question"]}
    <assistant>: {data_point["answer"]}
    """.strip()


def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    encoding = tokenizer(full_prompt, padding=True, truncation=True)
    return encoding

In [22]:
data = data["train"].shuffle(seed=42).map(generate_and_tokenize_prompt)
data

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 79
})

# Training

In [23]:
OUTPUT_DIR = "experiments"

In [ ]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs

Reusing TensorBoard on port 6006 (pid 2672), started 0:28:22 ago. (Use '!kill 2672' to kill it.)

In [32]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=80,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


model.config.use_cache = False
trainer.train()

  0%|          | 0/80 [00:00<?, ?it/s]

{'loss': 1.6185, 'grad_norm': 11.693669319152832, 'learning_rate': 5e-05, 'epoch': 0.05}
{'loss': 1.2714, 'grad_norm': 9.237638473510742, 'learning_rate': 0.0001, 'epoch': 0.1}
{'loss': 1.2099, 'grad_norm': 9.530993461608887, 'learning_rate': 0.00015000000000000001, 'epoch': 0.15}
{'loss': 0.9775, 'grad_norm': 8.32275104522705, 'learning_rate': 0.0002, 'epoch': 0.2}
{'loss': 1.0562, 'grad_norm': 9.168791770935059, 'learning_rate': 0.0001999145758387301, 'epoch': 0.25}
{'loss': 1.5148, 'grad_norm': 14.634392738342285, 'learning_rate': 0.000199658449300667, 'epoch': 0.3}
{'loss': 1.2372, 'grad_norm': 11.35930347442627, 'learning_rate': 0.0001992320579737045, 'epoch': 0.35}
{'loss': 1.3619, 'grad_norm': 14.262886047363281, 'learning_rate': 0.00019863613034027224, 'epoch': 0.41}
{'loss': 1.8435, 'grad_norm': 15.153573989868164, 'learning_rate': 0.00019787168453273544, 'epoch': 0.46}
{'loss': 1.5657, 'grad_norm': 10.735692977905273, 'learning_rate': 0.00019694002659393305, 'epoch': 0.51}
{'

TrainOutput(global_step=80, training_loss=0.8404941655695438, metrics={'train_runtime': 93.0161, 'train_samples_per_second': 3.44, 'train_steps_per_second': 0.86, 'total_flos': 97126647545856.0, 'train_loss': 0.8404941655695438, 'epoch': 4.0})

In [33]:
model.save_pretrained("trained-model")

# Load the model

In [12]:
# load the model from the disk in 'trained-model'

DEVICE = "cuda:0"
MODEL_NAME = "meta-llama/Llama-3.2-1B"
model = AutoModelForCausalLM.from_pretrained("trained-model").to(DEVICE)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

In [13]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [14]:
%%time

prompt = f"""
<human>: How can I create an account?
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
with torch.inference_mode():
    outputs = model. generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True) )

<human>: How can I create an account?
<assistant>: To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process. You can also sign in with your existing account if you already have an account.
CPU times: total: 734 ms
Wall time: 2.55 s


In [15]:
def generate_response(question: str) -> str:
    prompt = f"""
        <human>: {question}
        <assistant>:
        """.strip()
    encoding = tokenizer(prompt, return_tensors="pt") .to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) : ].strip()

In [18]:
prompt = "can a user write a review?"
print(generate_response(prompt))

Yes, a user can write a review on our website. Reviews help other users make informed decisions about our products. Please ensure your review is constructive and helpful.
